In [ ]:
import os
import shutil
import random
import torch

In [ ]:
# Run in a Colab cell
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.54 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (11990/11990), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:0

In [ ]:
# Install required packages
!pip install torch torchvision
!pip install opencv-python-headless
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch
from matplotlib import pyplot as plt
from PIL import Image

# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/test_model/best030422.pt', force_reload=True)

In [ ]:
# Inference
img_path = '/content/drive/MyDrive/test_model/tennis_ball.jpg'
results = model(img_path)

# Show results
results.print()
results.show()  # Opens image with detections

# Display inline
plt.imshow(Image.open(img_path))
plt.axis('off')
plt.show()

In [ ]:
!pip install transformers accelerate torchvision

In [ ]:
# 3D kalman filter trajectory code with depth estimation using MiDaS_small

import cv2
import torch
import numpy as np
import json
import argparse
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from models.common import DetectMultiBackend
from utils.dataloaders import LoadImages
from utils.general import (check_img_size, non_max_suppression, xyxy2xywh, increment_path)
from utils.torch_utils import select_device

# Install required packages for Google Colab
try:
    import transformers
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'transformers'])
    import transformers

def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    if ratio_pad is None:
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]
    coords[:, [1, 3]] -= pad[1]
    coords[:, :4] /= gain
    return coords

def clip_coords(boxes, shape):
    if isinstance(boxes, torch.Tensor):
        boxes[:, 0].clamp_(0, shape[1])
        boxes[:, 1].clamp_(0, shape[0])
        boxes[:, 2].clamp_(0, shape[1])
        boxes[:, 3].clamp_(0, shape[0])
    else:
        boxes[:, [0, 2]] = boxes[:, [0, 2]].clip(0, shape[1])
        boxes[:, [1, 3]] = boxes[:, [1, 3]].clip(0, shape[0])

# Depth estimation using MiDaS model
class DepthEstimator:
    def __init__(self, device='cpu'):
        self.device = device
        # Load MiDaS model for depth estimation
        self.model = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
        self.model.to(device)
        self.model.eval()

        # Load transforms
        midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
        self.transform = midas_transforms.small_transform

    def estimate_depth(self, image):
        """Estimate depth map from RGB image"""
        # Preprocess image
        input_batch = self.transform(image).to(self.device)

        # Predict depth
        with torch.no_grad():
            prediction = self.model(input_batch)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=image.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

        # Convert to numpy and normalize
        depth_map = prediction.cpu().numpy()
        return depth_map

    def get_depth_at_point(self, depth_map, x, y):
        """Get depth value at specific pixel coordinates"""
        if 0 <= y < depth_map.shape[0] and 0 <= x < depth_map.shape[1]:
            return depth_map[int(y), int(x)]
        return 0

# 3D Kalman Filter with gravity modeling
class KalmanFilter3D:
    def __init__(self, x, y, z, dt=1.0, g=0.5):
        self.dt = dt
        self.g = g
        # State: [x, y, z, vx, vy, vz]
        self.state = np.array([x, y, z, 0, 0, 0], dtype=np.float32)

        # State transition matrix (6x6)
        self.F = np.array([
            [1, 0, 0, dt, 0, 0],
            [0, 1, 0, 0, dt, 0],
            [0, 0, 1, 0, 0, dt],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]
        ], dtype=np.float32)

        # Measurement matrix (3x6) - we observe x, y, z
        self.H = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0]
        ], dtype=np.float32)

        # Process noise covariance (6x6)
        self.Q = np.eye(6, dtype=np.float32) * 0.01

        # Measurement noise covariance (3x3)
        self.R = np.array([
            [10, 0, 0],
            [0, 10, 0],
            [0, 0, 50]  # Higher noise for depth
        ], dtype=np.float32)

        # State covariance (6x6)
        self.P = np.eye(6, dtype=np.float32) * 100

    def predict(self):
        # State prediction
        self.state = self.F @ self.state
        # Apply gravity effects (assuming y is vertical axis)
        self.state[1] += 0.5 * self.g * self.dt**2  # Update y position
        self.state[4] += self.g * self.dt           # Update y velocity
        # Covariance prediction
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.state[0], self.state[1], self.state[2]

    def correct(self, x, y, z):
        z_measurement = np.array([x, y, z], dtype=np.float32)
        # Measurement residual
        y_residual = z_measurement - self.H @ self.state
        # Residual covariance
        S = self.H @ self.P @ self.H.T + self.R
        # Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(S)
        # State update
        self.state = self.state + K @ y_residual
        # Covariance update
        I = np.eye(6, dtype=np.float32)
        self.P = (I - K @ self.H) @ self.P

def run(
    weights,
    source,
    data,
    imgsz,
    conf_thres,
    iou_thres,
    max_det,
    device,
    project,
    name,
    exist_ok,
    half,
    dnn,
    g=0.5,
    early_pred_frame=10,
    depth_scale=1000.0
):
    source = str(source)
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)
    save_dir.mkdir(parents=True, exist_ok=True)

    # Initialize data collection
    actual_trajectory_3d = []
    predicted_full_trajectory_3d = []
    frame_height = None
    triggered = False

    # Device and model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)

    # Initialize depth estimator
    print("Loading depth estimation model...")
    depth_estimator = DepthEstimator(device=device)
    print("Depth estimation model loaded successfully!")

    # Dataset
    dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
    kf_3d = None

    # Process video
    for frame_idx, (path, im, im0s, vid_cap, s) in enumerate(dataset):
        print(f"Processing frame {frame_idx}")

        if frame_height is None:
            frame_height = im0s.shape[0]

        # Estimate depth map
        depth_map = depth_estimator.estimate_depth(im0s)

        # Preprocess for object detection
        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()
        im /= 255
        if len(im.shape) == 3:
            im = im[None]

        # Inference
        pred = model(im)
        pred = non_max_suppression(pred, conf_thres, iou_thres, None, False, max_det=max_det)

        # Process detections
        det = pred[0]
        current_point_3d = None

        if len(det):
            # Rescale boxes to original image
            det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0s.shape).round()
            # Get center of first detection
            *xyxy, conf, cls = det[0]
            cx = int((xyxy[0] + xyxy[2]) / 2)
            cy = int((xyxy[1] + xyxy[3]) / 2)

            # Get depth at detection center
            depth_value = depth_estimator.get_depth_at_point(depth_map, cx, cy)
            # Convert depth to real-world units (scaled)
            cz = depth_value / depth_scale

            # Initialize or update 3D Kalman Filter
            if kf_3d is None:
                kf_3d = KalmanFilter3D(cx, cy, cz, g=g)
                current_point_3d = (cx, cy, cz)
            else:
                # Predict before correction
                kf_3d.predict()
                kf_3d.correct(cx, cy, cz)
                current_point_3d = (int(kf_3d.state[0]), int(kf_3d.state[1]), kf_3d.state[2])

        # Handle no detection with existing KF
        elif kf_3d is not None:
            px, py, pz = kf_3d.predict()
            current_point_3d = (int(px), int(py), pz)

        # Store actual trajectory point
        if current_point_3d:
            actual_trajectory_3d.append(current_point_3d)
        else:
            actual_trajectory_3d.append(None)

        # Trigger full 3D prediction at specified frame
        if frame_idx == early_pred_frame and kf_3d is not None and not triggered:
            triggered = True
            x0, y0, z0, vx0, vy0, vz0 = kf_3d.state
            # Simulate 3D trajectory until ball hits bottom of frame
            for t in range(100):  # Max 100 frames prediction
                x = x0 + vx0 * t
                y = y0 + vy0 * t + 0.5 * g * t**2
                z = z0 + vz0 * t
                # Stop when ball hits ground
                if y >= frame_height - 5:  # 5px buffer
                    break
                predicted_full_trajectory_3d.append((int(x), int(y), z))

    # Save trajectory data
    trajectory_data = {
        'actual_3d': actual_trajectory_3d,
        'predicted_full_3d': predicted_full_trajectory_3d,
        'trigger_frame': early_pred_frame,
        'frame_height': frame_height,
        'depth_scale': depth_scale
    }
    with open(str(save_dir / 'trajectory_data_3d.json'), 'w') as f:
        json.dump(trajectory_data, f, default=str)  # Handle numpy types

    # Generate and save 3D plot
    plot_trajectory_3d(actual_trajectory_3d, predicted_full_trajectory_3d, frame_height, save_dir)

    print(f"Results saved to {save_dir}")

def plot_trajectory_3d(actual, predicted, frame_height, save_dir):
    fig = plt.figure(figsize=(15, 10))

    # 3D plot
    ax1 = fig.add_subplot(221, projection='3d')

    # Process actual trajectory
    actual_points = [p for p in actual if p is not None]
    if actual_points:
        actual_x = [p[0] for p in actual_points]
        actual_y = [frame_height - p[1] for p in actual_points]  # Flip Y for display
        actual_z = [p[2] for p in actual_points]

        ax1.plot(actual_x, actual_z, actual_y, 'go-', linewidth=2, markersize=4, label='Actual 3D Trajectory')

    # Process predicted trajectory
    if predicted:
        pred_x = [p[0] for p in predicted]
        pred_y = [frame_height - p[1] for p in predicted]  # Flip Y for display
        pred_z = [p[2] for p in predicted]

        ax1.plot(pred_x, pred_z, pred_y, 'r--', linewidth=2, label='Predicted 3D Trajectory')

    # Mark trigger point
    if actual_points and len(actual_points) > 10:
        trigger_point = actual_points[10]
        ax1.scatter([trigger_point[0]], [trigger_point[2]], [frame_height - trigger_point[1]],
                   c='blue', s=100, label='Prediction Trigger')

    ax1.set_xlabel('X Position (pixels)')
    ax1.set_ylabel('Z Position (depth units)')
    ax1.set_zlabel('Height from Bottom (pixels)')
    ax1.set_title('3D Ball Trajectory Prediction')
    ax1.legend()

    # XY projection
    ax2 = fig.add_subplot(222)
    if actual_points:
        ax2.plot(actual_x, actual_y, 'go-', linewidth=2, markersize=4, label='Actual XY')
    if predicted:
        ax2.plot(pred_x, pred_y, 'r--', linewidth=2, label='Predicted XY')
    ax2.set_xlabel('X Position (pixels)')
    ax2.set_ylabel('Height from Bottom (pixels)')
    ax2.set_title('XY Projection')
    ax2.legend()
    ax2.grid(True)

    # XZ projection
    ax3 = fig.add_subplot(223)
    if actual_points:
        ax3.plot(actual_x, actual_z, 'go-', linewidth=2, markersize=4, label='Actual XZ')
    if predicted:
        ax3.plot(pred_x, pred_z, 'r--', linewidth=2, label='Predicted XZ')
    ax3.set_xlabel('X Position (pixels)')
    ax3.set_ylabel('Z Position (depth units)')
    ax3.set_title('XZ Projection')
    ax3.legend()
    ax3.grid(True)

    # YZ projection
    ax4 = fig.add_subplot(224)
    if actual_points:
        ax4.plot(actual_z, actual_y, 'go-', linewidth=2, markersize=4, label='Actual YZ')
    if predicted:
        ax4.plot(pred_z, pred_y, 'r--', linewidth=2, label='Predicted YZ')
    ax4.set_xlabel('Z Position (depth units)')
    ax4.set_ylabel('Height from Bottom (pixels)')
    ax4.set_title('YZ Projection')
    ax4.legend()
    ax4.grid(True)

    # Save plot
    plot_path = save_dir / 'trajectory_plot_3d.png'
    plt.tight_layout()
    plt.savefig(str(plot_path), dpi=300, bbox_inches='tight')
    plt.close()
    print(f"3D Trajectory plot saved to {plot_path}")

def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', type=str, default='yolov5s.pt', help='model path')
    parser.add_argument('--source', type=str, required=True, help='video file path')
    parser.add_argument('--data', type=str, default='data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--imgsz', '--img-size', nargs='+', type=int, default=[640], help='inference size')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections')
    parser.add_argument('--device', default='', help='cuda device or cpu')
    parser.add_argument('--project', default='runs/detect', help='save results to project')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN')
    parser.add_argument('--g', type=float, default=0.5, help='gravity constant (pixels/frame^2)')
    parser.add_argument('--early-pred-frame', type=int, default=10, help='frame to trigger prediction')
    parser.add_argument('--depth-scale', type=float, default=1000.0, help='depth scaling factor')
    opt = parser.parse_args()
    opt.imgsz = opt.imgsz[0] if len(opt.imgsz) == 1 else opt.imgsz
    return opt

def main(opt):
    run(
        weights=opt.weights,
        source=opt.source,
        data=opt.data,
        imgsz=opt.imgsz,
        conf_thres=opt.conf_thres,
        iou_thres=opt.iou_thres,
        max_det=opt.max_det,
        device=opt.device,
        project=opt.project,
        name=opt.name,
        exist_ok=opt.exist_ok,
        half=opt.half,
        dnn=opt.dnn,
        g=opt.g,
        early_pred_frame=opt.early_pred_frame,
        depth_scale=opt.depth_scale
    )

if __name__ == "__main__":
    opt = parse_opt()
    main(opt)

In [ ]:
# 3D kalman filter trajectory code with depth estimation using Depth Anything V2

import cv2
import torch
import numpy as np
import json
import argparse
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import torch.nn.functional as F
from models.common import DetectMultiBackend
from utils.dataloaders import LoadImages
from utils.general import (check_img_size, non_max_suppression, xyxy2xywh, increment_path)
from utils.torch_utils import select_device

import cv2
import torch
import numpy as np
import json
import argparse
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import torch.nn.functional as F
from models.common import DetectMultiBackend
from utils.dataloaders import LoadImages
from utils.general import (check_img_size, non_max_suppression, xyxy2xywh, increment_path)
from utils.torch_utils import select_device

class DepthAnythingV2Estimator:
    def __init__(self, model_size='small'):
        self.model_size = model_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_configs = {
            'small': 'depth-anything/Depth-Anything-V2-Small-hf',
            'base': 'depth-anything/Depth-Anything-V2-Base-hf',
            'large': 'depth-anything/Depth-Anything-V2-Large-hf'
        }
        try:
            from transformers import pipeline
            model_name = self.model_configs.get(model_size, self.model_configs['base'])
            print(f"Loading Depth Anything V2 {model_size} model...")
            self.pipe = pipeline(
                task="depth-estimation",
                model=model_name,
                device=0 if torch.cuda.is_available() else -1
            )
            self.method = f'depth_anything_v2_{model_size}'
            print(f"Depth Anything V2 {model_size} loaded successfully")
        except Exception as e:
            print(f"Failed to load Depth Anything V2: {e}")
            self.pipe = None
            self.method = 'simple'

    def preprocess_image(self, image):
        if isinstance(image, np.ndarray):
            if len(image.shape) == 3 and image.shape[2] == 3:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image.astype(np.uint8))
        return image

    def estimate_depth(self, frame, bbox=None):
        if self.pipe is not None:
            try:
                image = self.preprocess_image(frame)
                result = self.pipe(image)
                depth_map = np.array(result['depth'])
                depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
                depth_map = depth_map * 255

                if bbox is not None:
                    x1, y1, x2, y2 = map(int, bbox)
                    h, w = depth_map.shape[:2]
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(w, x2), min(h, y2)
                    if x2 > x1 and y2 > y1:
                        roi_depth = depth_map[y1:y2, x1:x2]
                        if roi_depth.size > 0:
                            median_depth = float(np.median(roi_depth))
                            depth_distance = max(10.0, min(500.0, 300.0 - median_depth))
                            return depth_distance
                    return 100.0

                return depth_map

            except Exception as e:
                print(f"Depth estimation failed: {e}")
                return 100.0
        else:
            return 100.0

class KalmanFilter3D:
    def __init__(self, x, y, z, dt=1.0, g=0.5):
        self.dt = dt
        self.g = g
        self.state = np.array([x, y, z, 0, 0, 0], dtype=np.float32)
        self.F = np.array([
            [1, 0, 0, dt, 0, 0],
            [0, 1, 0, 0, dt, 0],
            [0, 0, 1, 0, 0, dt],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]
        ], dtype=np.float32)
        self.H = np.eye(3, 6, dtype=np.float32)
        self.Q = np.eye(6, dtype=np.float32) * 0.01
        self.R = np.diag([10, 10, 50]).astype(np.float32)
        self.P = np.eye(6, dtype=np.float32) * 100

    def predict(self):
        self.state = self.F @ self.state
        self.state[1] += 0.5 * self.g * self.dt**2
        self.state[4] += self.g * self.dt
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.state[0], self.state[1], self.state[2]

    def correct(self, x, y, z):
        z_measure = np.array([x, y, z], dtype=np.float32)
        y_residual = z_measure - self.H @ self.state
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.state = self.state + K @ y_residual
        I = np.eye(6, dtype=np.float32)
        self.P = (I - K @ self.H) @ self.P

def plot_trajectory_3d(actual, predicted, frame_height, save_dir):
    fig = plt.figure(figsize=(15, 10))
    ax1 = fig.add_subplot(221, projection='3d')
    actual_points = [p for p in actual if p is not None]
    if actual_points:
        actual_x = [p[0] for p in actual_points]
        actual_y = [frame_height - p[1] for p in actual_points]
        actual_z = [p[2] for p in actual_points]
        ax1.plot(actual_x, actual_z, actual_y, 'go-', linewidth=2, markersize=4, label='Actual 3D Trajectory')
    if predicted:
        pred_x = [p[0] for p in predicted]
        pred_y = [frame_height - p[1] for p in predicted]
        pred_z = [p[2] for p in predicted]
        ax1.plot(pred_x, pred_z, pred_y, 'r--', linewidth=2, label='Predicted 3D Trajectory')
    ax1.set_xlabel('X Position (pixels)')
    ax1.set_ylabel('Z Position (depth units)')
    ax1.set_zlabel('Height from Bottom (pixels)')
    ax1.set_title('3D Ball Trajectory Prediction')
    ax1.legend()
    ax2 = fig.add_subplot(222)
    if actual_points:
        ax2.plot(actual_x, actual_y, 'go-', linewidth=2, markersize=4, label='Actual XY')
    if predicted:
        ax2.plot(pred_x, pred_y, 'r--', linewidth=2, label='Predicted XY')
    ax2.set_xlabel('X Position (pixels)')
    ax2.set_ylabel('Height from Bottom (pixels)')
    ax2.set_title('XY Projection')
    ax2.legend()
    ax2.grid(True)
    ax3 = fig.add_subplot(223)
    if actual_points:
        ax3.plot(actual_x, actual_z, 'go-', linewidth=2, markersize=4, label='Actual XZ')
    if predicted:
        ax3.plot(pred_x, pred_z, 'r--', linewidth=2, label='Predicted XZ')
    ax3.set_xlabel('X Position (pixels)')
    ax3.set_ylabel('Z Position (depth units)')
    ax3.set_title('XZ Projection')
    ax3.legend()
    ax3.grid(True)
    ax4 = fig.add_subplot(224)
    if actual_points:
        ax4.plot(actual_z, actual_y, 'go-', linewidth=2, markersize=4, label='Actual YZ')
    if predicted:
        ax4.plot(pred_z, pred_y, 'r--', linewidth=2, label='Predicted YZ')
    ax4.set_xlabel('Z Position (depth units)')
    ax4.set_ylabel('Height from Bottom (pixels)')
    ax4.set_title('YZ Projection')
    ax4.legend()
    ax4.grid(True)
    plt.tight_layout()
    plt.savefig(str(save_dir / 'trajectory_plot_3d.png'), dpi=300, bbox_inches='tight')
    plt.close()
    print(f"3D Trajectory plot saved to {save_dir / 'trajectory_plot_3d.png'}")

def run(weights, source, data, imgsz, conf_thres, iou_thres, max_det, device, project, name, exist_ok, half, dnn, g, early_pred_frame):
    source = str(source)
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)
    save_dir.mkdir(parents=True, exist_ok=True)

    actual_trajectory_3d = []
    predicted_full_trajectory_3d = []
    frame_height = None
    triggered = False

    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)

    depth_estimator = DepthAnythingV2Estimator(model_size='small')

    dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
    kf_3d = None

    for frame_idx, (path, im, im0s, vid_cap, s) in enumerate(dataset):
        print(f"Processing frame {frame_idx}")
        if frame_height is None:
            frame_height = im0s.shape[0]

        im = torch.from_numpy(im).to(device)
        im = im.half() if half else im.float()
        im /= 255
        if len(im.shape) == 3:
            im = im[None]

        pred = model(im)
        pred = non_max_suppression(pred, conf_thres, iou_thres, None, False, max_det=max_det)

        det = pred[0]
        current_point_3d = None

        if len(det):
            det[:, :4] = det[:, :4].round()
            *xyxy, conf, cls = det[0]
            cx = int((xyxy[0] + xyxy[2]) / 2)
            cy = int((xyxy[1] + xyxy[3]) / 2)
            bbox = (int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]))
            cz = depth_estimator.estimate_depth(im0s, bbox)

            if kf_3d is None:
                kf_3d = KalmanFilter3D(cx, cy, cz, g=g)
                current_point_3d = (cx, cy, cz)
            else:
                kf_3d.predict()
                kf_3d.correct(cx, cy, cz)
                current_point_3d = (int(kf_3d.state[0]), int(kf_3d.state[1]), kf_3d.state[2])

        elif kf_3d is not None:
            px, py, pz = kf_3d.predict()
            current_point_3d = (int(px), int(py), pz)

        if current_point_3d:
            actual_trajectory_3d.append(current_point_3d)
        else:
            actual_trajectory_3d.append(None)

        if frame_idx == early_pred_frame and kf_3d is not None and not triggered:
            triggered = True
            x0, y0, z0, vx0, vy0, vz0 = kf_3d.state
            for t in range(100):
                x = x0 + vx0 * t
                y = y0 + vy0 * t + 0.5 * g * t**2
                z = z0 + vz0 * t
                if y >= frame_height - 5:
                    break
                predicted_full_trajectory_3d.append((int(x), int(y), z))

    with open(str(save_dir / 'trajectory_data_3d.json'), 'w') as f:
        json.dump({
            'actual_3d': actual_trajectory_3d,
            'predicted_full_3d': predicted_full_trajectory_3d,
            'trigger_frame': early_pred_frame,
            'frame_height': frame_height
        }, f, default=str)

    plot_trajectory_3d(actual_trajectory_3d, predicted_full_trajectory_3d, frame_height, save_dir)
    print(f"Results saved to {save_dir}")

def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', type=str, default='yolov5s.pt')
    parser.add_argument('--source', type=str, required=True)
    parser.add_argument('--data', type=str, default='data/coco128.yaml')
    parser.add_argument('--imgsz', nargs='+', type=int, default=[640])
    parser.add_argument('--conf-thres', type=float, default=0.25)
    parser.add_argument('--iou-thres', type=float, default=0.45)
    parser.add_argument('--max-det', type=int, default=1000)
    parser.add_argument('--device', default='')
    parser.add_argument('--project', default='runs/detect')
    parser.add_argument('--name', default='exp')
    parser.add_argument('--exist-ok', action='store_true')
    parser.add_argument('--half', action='store_true')
    parser.add_argument('--dnn', action='store_true')
    parser.add_argument('--g', type=float, default=0.5)
    parser.add_argument('--early-pred-frame', type=int, default=10)
    opt = parser.parse_args()
    opt.imgsz = opt.imgsz[0] if len(opt.imgsz) == 1 else opt.imgsz
    return opt

def main(opt):
    run(**vars(opt))

if __name__ == "__main__":
    opt = parse_opt()
    main(opt)


In [ ]:
!python detect.py \
--weights /content/drive/MyDrive/test_model/best030422.pt \
--source /content/drive/MyDrive/test_model/t4.mp4 \
--project /content/drive/MyDrive/Finalresults \
--name trajectory_results4 \
--conf-thres 0.3 \
--early-pred-frame 10 \
--device cpu